sqldatabase chain is the class that represents a SQL database chain in the LangChain framework. It extends the BaseChain class and implements the functionality specific to a SQL database chain.

In [1]:
 
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3", temperature=1)

In [3]:
from langchain_community.utilities import SQLDatabase


# MySQL connection URI
username = 'root'
password = 'manisha'
host = '127.0.0.1'
dbname = 'test'  # Database name

# Constructing the MySQL URI
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}/{dbname}"

# # Initializing SQLDatabase object for MySQL
db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE transaction2 (
	account_number VARCHAR(255) NOT NULL, 
	transaction_details TEXT, 
	withdrawal_amount DECIMAL(10, 2), 
	deposit_amount DECIMAL(10, 2), 
	balance_amount DECIMAL(10, 2), 
	value_date DATETIME, 
	transactions_category TEXT, 
	id INTEGER NOT NULL AUTO_INCREMENT, 
	PRIMARY KEY (id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from transaction2 table:
account_number	transaction_details	withdrawal_amount	deposit_amount	balance_amount	value_date	transactions_category	id
409000611074	TRF FROM  Indiaforensic SERVICES	0.00	1000000.00	1000000.00	2022-11-16 00:00:00	food	1
409000611074	TRF FROM  Indiaforensic SERVICES	0.00	1000000.00	2000000.00	2022-11-22 00:00:00	medical	2
409000611074	FDRL/INTERNAL FUND TRANSFE	0.00	500000.00	2500000.00	2022-12-05 00:00:00	groceries	3
*/


CREATE TABLE transactions (
	`Account_No` VARCHAR(50) NOT NULL, 
	`Transaction_details` TEXT, 
	`Withdrawal_amount` INTEGER, 
	`Deposit_amount` INTEGER, 
	`Balance_a

In [6]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)
qns1=db_chain("What was my total spending last week?  ")


c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
What was my total spending last week?  
SQLQuery:SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE YEAR(Value_date) = YEAR(CURDATE()) AND WEEK(Value_date) = WEEK(CURDATE()) - 1 GROUP BY Account_No LIMIT 5
SQLResult: [(Decimal('72443'),), (Decimal('14900000'),), (Decimal('188891'),), (Decimal('3400000'),), (Decimal('16494712'),)]
Answer:Question: What was my total spending last week?  
SQLQuery: SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE YEAR(Value_date) = YEAR(CURDATE()) AND WEEK(Value_date) = WEEK(CURDATE()) - 1 GROUP BY Account_No LIMIT 5
> Finished chain.


In [8]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)
qns1=db_chain("What was my total spending previous week?my account number is:409000611074. ")
# did few shot learning



> Entering new SQLDatabaseChain chain...
What was my total spending previous week?my account number is:409000611074. 
SQLQuery:SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE Account_No = '409000611074' AND Value_date >= DATE_SUB(CURDATE(), INTERVAL 1 WEEK) GROUP BY Account_No LIMIT 1
SQLResult: 
Answer:Question: What was my total spending previous week?my account number is:409000611074. 

SQLQuery:SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE Account_No = '409000611074' AND Value_date >= DATE_SUB(CURDATE(), INTERVAL 1 WEEK) GROUP BY Account_No LIMIT 1
> Finished chain.


In [41]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)
qns1=db_chain("How many transactions were below $100 this year ?")
# right answer



> Entering new SQLDatabaseChain chain...
How many transactions were below $100 this year ?
SQLQuery:SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE YEAR(Value_date) = YEAR(CURDATE()) AND Withdrawal_amount < 100 OR Deposit_amount < 100
SQLResult: [(61752,)]
Answer:Here is the answer:

Question: How many transactions were below $100 this year ?
SQLQuery: SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE YEAR(Value_date) = YEAR(CURDATE()) AND (Withdrawal_amount < 100 OR Deposit_amount < 100
> Finished chain.


In [43]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)
qns1=db_chain("give me the average spending of mine each month this year?my account number is 409000611074. ")
# right 




> Entering new SQLDatabaseChain chain...
give me the average spending of mine each month this year?my account number is 409000611074. 
SQLQuery:SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, AVG(Withdrawal_amount) AS Average_Spending                   FROM transactions                   WHERE Account_No = '409000611074' AND YEAR(Value_date) = YEAR(CURDATE())                   GROUP BY YEAR(Value_date), MONTH(Value_date)                   ORDER BY YEAR(Value_date), MONTH(Value_date) DESC                   LIMIT 12
SQLResult: [(2024, 6, Decimal('126570.7719')), (2024, 5, Decimal('132303.3111')), (2024, 4, Decimal('118347.4944')), (2024, 3, Decimal('106402.8506')), (2024, 2, Decimal('126567.5769')), (2024, 1, Decimal('138482.5875'))]
Answer:Here is the answer:

Question: give me the average spending of mine each month this year?my account number is 409000611074.
SQLQuery: SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, AVG(Withdrawal_amount) AS Average_Spendin

In [44]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)
qns1=db_chain("How much did I save last month?my account number is 409000611074.")
# this month saving shown



> Entering new SQLDatabaseChain chain...
How much did I save last month?my account number is 409000611074.
SQLQuery:SELECT SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings FROM transactions WHERE Account_No = '409000611074' AND YEAR(Value_date) = YEAR(CURDATE()) AND MONTH(Value_date) = MONTH(CURDATE()) ORDER BY Value_date DESC LIMIT 1
SQLResult: [(None,)]
Answer:Here is the answer:

Question: How much did I save last month?my account number is 409000611074.
SQLQuery: SELECT SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings FROM transactions WHERE Account_No = '409000611074' AND YEAR(Value_date) = YEAR(CURDATE()) AND MONTH(Value_date) = MONTH(CURDATE()) ORDER BY Value_date DESC LIMIT 1
> Finished chain.


In [48]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)
qns1=db_chain("What was my total income last month?my account number is 409000611074.")

# right



> Entering new SQLDatabaseChain chain...
What was my total income last month?my account number is 409000611074.
SQLQuery:SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = '409000611074' AND YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) - 1
SQLResult: [(Decimal('5900000'),)]
Answer:Question: What was my total income last month?my account number is 409000611074.
SQLQuery: SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = '409000611074' AND YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) - 1
> Finished chain.


In [49]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)
qns1=db_chain("How many transactions did I make last month?my account number is 409000611074.")
# right



> Entering new SQLDatabaseChain chain...
How many transactions did I make last month?my account number is 409000611074.
SQLQuery:SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE Account_No = '409000611074' AND YEAR(Value_date) = YEAR(CURDATE()) AND MONTH(Value_date) = MONTH(CURDATE()) - 1
SQLResult: [(57,)]
Answer:Here is the answer:

Question: How many transactions did I make last month?my account number is 409000611074.
SQLQuery:SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE Account_No = '409000611074' AND YEAR(Value_date) = YEAR(CURDATE()) AND MONTH(Value_date) = MONTH(CURDATE()) - 1
> Finished chain.


In [9]:

qns1=db_chain.run("What is my consistent income in my last 3 months?my account number is 409000611074.")
# few shot learning

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
What is my consistent income in my last 3 months?my account number is 409000611074.
SQLQuery:SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = '409000611074' AND Value_date >= DATE_SUB(CURDATE(), INTERVAL 3 MONTH) GROUP BY YEAR(Value_date), MONTH(Value_date) ORDER BY Total_Income DESC LIMIT 1
SQLResult: [(Decimal('12100000'),)]
Answer:Question: What is my consistent income in my last 3 months?my account number is 409000611074.

SQLQuery: SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = '409000611074' AND Value_date >= DATE_SUB(CURDATE(), INTERVAL 3 MONTH) GROUP BY YEAR(Value_date), MONTH(Value_date) ORDER BY Total_Income DESC LIMIT 1
> Finished chain.


In [64]:

qns1=db_chain("What are my recent deposits and withdrawals?my account number is 409000611074.")
# right



> Entering new SQLDatabaseChain chain...
What are my recent deposits and withdrawals?my account number is 409000611074.
SQLQuery:SELECT Withdrawal_amount AS Withdrawals, Deposit_amount AS Deposits FROM transactions WHERE Account_No = '409000611074' ORDER BY Value_date DESC LIMIT 5
SQLResult: [(600000, 0), (132851, 0), (720, 0), (0, 300000), (224051, 0)]
Answer:Here is the answer:

Question: What are my recent deposits and withdrawals?my account number is 409000611074.
SQLQuery:SELECT Withdrawal_amount AS Withdrawals, Deposit_amount AS Deposits FROM transactions WHERE Account_No = '409000611074' ORDER BY Value_date DESC LIMIT 5
> Finished chain.


In [10]:
result = db_chain(" what is my Income of last 3 months?my account number is 409000611074.")

# FEW SHORT LEARNING

In [54]:
#Few_shorts

few_shots=[
    {
        'Question':" Income of last 3 months. ?",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH) """,
        'SQLResult':"1530205868",
        'Answer':"1530205868 is the income of last 3 months."
    }
    ,

    {
        'Question':"Total expenses/spendings of last 8 months ?",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 8 MONTH) """,
        'SQLResult':"11274528407",
        'Answer':"11274528407 is the total expenses of last 8 months."
    }
    ,
      {
        'Question':"Total saving of last month as my account number is 409000493201'  ?",
        'SQLQuery':"""SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Savings_Last_Month FROM transactions WHERE Account_No = "409000493201'" AND YEAR(Value_date) = YEAR(CURRENT_DATE() 
- INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH);
        """,
        'SQLResult':"-193509",
        'Answer':"You lost -193509 last month."
    }
,
    {
        'Question':"How many transactions did I make in last 2 week  ",
        'SQLQuery':"""SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 WEEK) """,
        'SQLResult':"162",
        'Answer':"You made 162 transactions in last 2 week."
    },
     {
        'Question':"Amount spent last week ",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 WEEK) """,
        'SQLResult':"",
        'Answer':"0 is your total spending last week."
    }
    , {
        'Question': "Expenses for each day this month ?",
        'SQLQuery': """SELECT Date, SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) GROUP BY Date """,
        'SQLResult':
        """
Date                        Total_Expenses
2024-06-02                   183648
2024-06-03                   80666
2024-06-04                   135504
2024-06-05                   92031
2024-06-06                   118475
2024-06-07                   16440
2024-06-09                   62112
2024-06-10                   10331
"""
        ,
        'Answer': """The total_expenses is 183648,80666,135504,92031,118475,16440,62112,10331 for the year 2024 month of 6 and date 02,03,04,05,06,07,09 and 10.  """         
    },
    {
        'Question':"savings of previous year for each month ",
        'SQLQuery':"""SELECT MONTH(Value_date) AS Month, (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Total_Savings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 YEAR) GROUP BY MONTH(Value_date) ORDER BY MONTH(Value_date) """,
        'SQLResult':"""
        Date                Total_Expenses
        1                       -311244
        2                          9101
        3                         3476          
        4                       -15971
        5                       322249
        6                       -172560
        7                       361471
        8                       -446844
        9                       93215
        10                      -73917
        11                      294502
        12                      -404028""",
        'Answer':"Month-1:-311244, Month-2:9101, Month-3:3476"
    },
     {
        'Question':"Amount Saved each year",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year,SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions GROUP BY YEAR(Value_date) """,
        'SQLResult':"""
        Year                Savings
        2021                 497766
        2022                 272458 
        2023                 -340550          
        2024                 -148114
       """,
        'Answer':"You saved 497766 in 2021 ,272458 in 2022, -340550 in 2023 and -148114 in 2024."
    }
    ,
     {
        'Question':"Savings of last 2 year  ",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Net_Savings FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 YEAR) GROUP BY YEAR(Value_date) """,
        'SQLResult':"""
        Year                Savings
        2022                 2270453
        2023                 -895783          
        2024                 -912470
        """,
        'Answer':"You saved 2270453 in 2022, -89578 in 2023 and -912470 in 2024."
    }
    ,
     {
        'Question':"Expenses of each day of last week  ",
        'SQLQuery':"""SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE Value_date >= DATE_SUB(CURDATE(), INTERVAL 1 WEEK) GROUP BY DATE(Value_date) """,
        'SQLResult':"""
        Year                Savings
        2024-06-06           118475
        2024-06-07           16440         
        2024-06-09           62112
        2024-06-10           10331

        """,
        'Answer':"You spend 118475,16440,62112,10331."
    },
    {
        'Question':"Amount spent this year ?",
        'SQLQuery':"""SELECT year(Value_date) as Year,sum(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE year(Value_date) = year(curdate()) group by year(Value_date) """,
        'SQLResult':"""
        Year                Total_Expenses
       2024                    6527128485
        """,
        'Answer':"You spent 36616174 this year."
    },
    {
        'Question':"amount/money saved each month previous year ?",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE - INTERVAL 1 YEAR) GROUP BY YEAR(Value_date), MONTH(Value_date) """,
        'SQLResult':"""
        Year            Month              Savings
       2023              1                  -311244
       2023             2                       9101
       2023             3                       3476    
       2023             4                       -15971
       2023             5                       322249
       2023             6                       -172560
        2023             7                   361471
       2023             8                       -446844
       2023             9                       93215    
       2023             10                       -73917
       2023            11                       294502
       2023             12                       -404028
        """,
        'Answer':"You lost 311244 in month 1 ,saved 9101,3476 in month 2 and month 3  previous year."
    }
     ,
      {
        'Question':"income of the previous year ",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE) - 1 """,
        'SQLResult':"38593214103",
        'Answer':"Your income previous year was 38593214103."
    }
    ,
      {
        'Question':"income of the first 4 month of previous year ",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE MONTH(Value_date) IN (1,2,3,4) AND YEAR(Value_date) = YEAR(CURRENT_DATE) - 1 """,
        'SQLResult':"17847505444",
        'Answer':"Your income of first 4 month of previous year was 17847505444."
    },
    {
    'Question': "amount I lost each year?",
    'SQLQuery': """
                SELECT YEAR(Value_date) AS Year,SUM(Withdrawal_amount) - SUM(Deposit_amount) AS Losses 
                FROM transactions 
                GROUP BY YEAR(Value_date);
                """,
    'SQLResult': """[
        (2022, Decimal('211175202')),
        (2023, Decimal('575324140')),
        (2024, Decimal('490771857')),
        (2021, Decimal('906602921')),
        (2020, Decimal('1076993185'))

    ]"""
    ,
    'Answer': "You lost 497766 in 2021, 272458 in 2022, -340550 in 2023, and -148114 in 2024."
},

{

    'Question':"How much I earned in total last month?", 
    'SQLQuery':"SELECT SUM(Deposit_amount) AS Total_Earnings FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)",
    'SQLResult':'Decimal(394498711))',
    'Answer': "You earned 394498711 last month."
},

{
    'Question':"what is my consistent income in last 3 months?my account number is 409000611074.", 
    'SQLQuery':"SELECT AVG(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = '409000611074' AND Value_date >= DATE_SUB(CURDATE(), INTERVAL 3 MONTH) GROUP BY YEAR(Value_date), MONTH(Value_date) ORDER BY Total_Income DESC LIMIT 1",
    'SQLResult':'Decimal(134444.4444))',
    'Answer': "The consistent income of last three months was 134444.4444 ."
},
{
    'Question':"What was my total spending previous week?my account number is 409000611074.", 
    'SQLQuery':"SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE Account_No = '409000611074' AND Value_date >= DATE_SUB(CURDATE(), INTERVAL 2 WEEK) AND Value_date < DATE_SUB(CURDATE(), INTERVAL 1 WEEK);",
    'SQLResult':'None',
    'Answer': "There is no any spending this week ."
}

]




In [55]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

In [56]:
to_vectorize 

[' Income of last 3 months. ? SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH)  1530205868 1530205868 is the income of last 3 months.',
 'Total expenses/spendings of last 8 months ? SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 8 MONTH)  11274528407 11274528407 is the total expenses of last 8 months.',
 'Total saving of last month as my account number is 409000493201\'  ? SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Savings_Last_Month FROM transactions WHERE Account_No = "409000493201\'" AND YEAR(Value_date) = YEAR(CURRENT_DATE() \n- INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH);\n         -193509 You lost -193509 last month.',
 'How many transactions did I make in last 2 week   SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 WE

In [57]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [60]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=1,
)
example_selector.select_examples({"Question": "What are my consistent Income of last 3 months?'?"})

[{'Answer': 'The consistent income of last three months was 134444.4444 .',
  'Question': 'what is my consistent income in last 3 months?my account number is 409000611074.',
  'SQLQuery': "SELECT AVG(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = '409000611074' AND Value_date >= DATE_SUB(CURDATE(), INTERVAL 3 MONTH) GROUP BY YEAR(Value_date), MONTH(Value_date) ORDER BY Total_Income DESC LIMIT 1",
  'SQLResult': 'Decimal(134444.4444))'}]

In [28]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX,_mysql_prompt 

In [29]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [14]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [15]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [16]:
suffix_eg="Question: {input}"

In [17]:
print(suffix_eg)

Question: {input}


In [18]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=suffix_eg,
    input_variables=["input", "top_k"], #These variables are used in the prefix and suffix
)

In [19]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)


In [20]:
print(few_shot_prompt)

input_variables=['input', 'top_k'] example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000029D672C0490>, k=1, example_keys=None, input_keys=None, vectorstore_kwargs=None) example_prompt=PromptTemplate(input_variables=['Answer', 'Question', 'SQLQuery', 'SQLResult'], template='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}') suffix='Question: {input}' prefix='\nYou are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer th

In [62]:
result = new_chain("what is my consistent income in last 3 months?my account number is 409000611074.")



> Entering new SQLDatabaseChain chain...
what is my consistent income in last 3 months?my account number is 409000611074.
SQLQuery:SELECT AVG(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = '409000611074' AND Value_date >= DATE_SUB(CURDATE(), INTERVAL 3 MONTH) GROUP BY YEAR(Value_date), MONTH(Value_date) ORDER BY Total_Income DESC LIMIT 1
SQLResult: [(Decimal('134444.4444'),)]
Answer:

KeyboardInterrupt: 